## Version 3
### accuracy: 0.97533
- only delete punctuation + jieba + remove stopwords
- Tokenizer
- TfidfVectorizer
- Ensemble model

### 改进方向
- 存在非中文content，训练集的去掉
- lightgbm参数调优，加入catboost分类器
- 融合 bert-like transformer(prob < x)

In [1]:
import pandas as pd
import string
import re
import jieba
import joblib
from tqdm import tqdm
tqdm.pandas()
from tokenizers import (
    models,
    normalizers,
    pre_tokenizers,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from transformers import PreTrainedTokenizerFast
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

In [2]:
train_dir = './train.csv'
test_dir = './test.csv'
df1 = pd.read_csv(train_dir)
df2 = pd.read_csv(test_dir)

In [6]:
train = df1[['id', 'content', 'topic']]
test = df2[['id', 'content', 'topic']]
train

,id,content,topic
0,108,当地时间11月17日上午，亚太经合组织第三十次领导人非正式会议在美国旧金山莫斯科尼中心举行。...,1
1,7874,推动中美关系重回正轨 领航亚太发展繁荣新程——国际社会期待习近平主席将赴美国举行中美元首会晤...,1
2,7388,9月26日，国务院新闻办公室发布《携手构建人类命运共同体：中国的倡议与行动》白皮书并举行新闻...,1
3,5271,亚太经合组织第二十七次领导人非正式会议是亚太经合组织最高级别的会议。亚太经合组织在成立的最初...,1
4,3781,当地时间11月17日上午，亚太经合组织第三十次领导人非正式会议在美国旧金山莫斯科尼中心举行。...,1
...,...,...,...
6805,2604,缅甸再起战火，地方武装袭击多个军事据点，宣称剿灭电信诈骗缅甸北部燃起的战火，牵动着中国人的目...,23
6806,3157,中国军队采取行动，前往中缅边境，给缅甸内战双方发出信号！缅甸内战如今越发激烈，中国和缅甸作为...,23
6807,1567,正文申请入驻缅甸内战在争夺什么？2023-12-13 20:47:40 来源:宋鸿兵北京举报...,23
6808,9353,切换模式写文章登录/注册中国和缅甸划清边界的时候，解放军还到缅甸打了一仗沈听雪喜欢军史党史的...,23


In [9]:
import requests

# DeepL API endpoint
DEEPL_API_URL = "https://api-free.deepl.com/v2/translate"

DEEPL_API_KEY = "67e30ce8-6c20-49a9-917b-fc252b83adff:fx"


def translate_text(text, target_lang="ch-ZH"):
    """
    使用DeepL API翻译文本

    Args:
        text (str): 需要翻译的中文文本
        target_lang (str): 目标语言代码，默认为英语"EN"

    Returns:
        str: 翻译后的文本
    """
    params = {
        "auth_key": DEEPL_API_KEY,
        "text": text,
        "target_lang": target_lang,
        "source_lang": "ZH"  # 指定源语言为中文
    }

    response = requests.post(DEEPL_API_URL, data=params)
    response_json = response.json()

    # 返回翻译结果
    return response_json["translations"][0]["text"] if response.status_code == 200 else None

In [4]:
non_chinese_content_train = train[~train['content'].str.contains(r'[\u4e00-\u9fff]')]
non_chinese_content_train

,id,content,topic
473,831,ChatGPT hastaken the world by stormsince it la...,2
2030,10028,%PDF-1.7\r\n%Â³ÇØ\r\n1 0 obj\r\n<> /Outlines 5...,9
2174,10036,%PDF-1.7\r\n%Â³ÇØ\r\n1 0 obj\r\n<> /Outlines 5...,9
2510,3061,"This war has been going on for a long time, an...",11
2562,3577,е…ЁйғЁеҜјиҲӘж—¶ж”ҝеӣҪеҶ…иҙўз»ҸзӨҫдјҡеЁұд№җеӨ©ж...,11
4863,9050,The robot Chenchen represents the Beijing-Hang...,16
5508,3706,зҷҫеәҰйҰ–йЎөе•ҶеҹҺжіЁеҶҢзҷ»еҪ•зҪ‘йЎөиө„и®Ҝи§Ҷй...,19


In [9]:
# 删除非中文content
train = train[train['content'].str.contains(r'[\u4e00-\u9fff]')]
train

,id,content,topic
0,108,当地时间11月17日上午，亚太经合组织第三十次领导人非正式会议在美国旧金山莫斯科尼中心举行。...,1
1,7874,推动中美关系重回正轨 领航亚太发展繁荣新程——国际社会期待习近平主席将赴美国举行中美元首会晤...,1
2,7388,9月26日，国务院新闻办公室发布《携手构建人类命运共同体：中国的倡议与行动》白皮书并举行新闻...,1
3,5271,亚太经合组织第二十七次领导人非正式会议是亚太经合组织最高级别的会议。亚太经合组织在成立的最初...,1
4,3781,当地时间11月17日上午，亚太经合组织第三十次领导人非正式会议在美国旧金山莫斯科尼中心举行。...,1
...,...,...,...
6805,2604,缅甸再起战火，地方武装袭击多个军事据点，宣称剿灭电信诈骗缅甸北部燃起的战火，牵动着中国人的目...,23
6806,3157,中国军队采取行动，前往中缅边境，给缅甸内战双方发出信号！缅甸内战如今越发激烈，中国和缅甸作为...,23
6807,1567,正文申请入驻缅甸内战在争夺什么？2023-12-13 20:47:40 来源:宋鸿兵北京举报...,23
6808,9353,切换模式写文章登录/注册中国和缅甸划清边界的时候，解放军还到缅甸打了一仗沈听雪喜欢军史党史的...,23


In [10]:
non_chinese_content_test = test[~test['content'].str.contains(r'[\u4e00-\u9fff]')]
non_chinese_content_test

,id,content,topic
123,8603,China aims to increase the share of electric v...,2
2723,6677,е…ЁйғЁеҜјиҲӘж—¶ж”ҝеӣҪеҶ…еӣҪйҷ…иҙўз»ҸзӨҫдјҡеЁұд...,18
2859,7296,China will launch the three astronauts of theS...,19


In [11]:
def get_stopword():
    with open('./baidu_stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = set(line.strip() for line in f)
    stopwords.update(["年","月","日"])
    return stopwords

def wordopt_cn(text):
    # replace punctuation with space
    no_punct = re.sub(pattern, ' ', text)
    return str(no_punct)

def remove_stopwords(words):
    return (word for word in words if word not in stopwords)

def process_content(content):
    content = wordopt_cn(content)
    words = jieba.cut(content)
    words = remove_stopwords(words)
    content = ' '.join(words)
    content = re.sub(spaces_pattern, ' ', content)
    return content

# Precompile the regular expressions
punctuation = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏." + string.punctuation
pattern = re.compile('[%s]' % re.escape(punctuation))
spaces_pattern = re.compile(r'\s+')
stopwords = get_stopword()

In [12]:
train.loc[:,'content'] = train['content'].progress_apply(process_content)
train.to_csv('train_jieba.csv', index=False)

  0%|          | 0/6803 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\shens\AppData\Local\Temp\jieba.cache
Loading model cost 0.626 seconds.
Prefix dict has been built successfully.
100%|██████████| 6803/6803 [00:29<00:00, 233.63it/s]


In [13]:
test.loc[:,'content'] = test['content'].progress_apply(process_content)
test.to_csv('test_jieba.csv', index=False)

100%|██████████| 3405/3405 [00:14<00:00, 235.07it/s]


In [14]:
test

,id,content,topic
0,181,2023 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人...,1
1,1763,当地 时间 11 16 习近平 主席 亚太经合组织 APEC 工商 领导人 峰会 发表 书面...,1
2,1015,亚太经合组织 第三十次 领导人 非正式 会议 美国 旧金山 莫斯科 尼 中心 国家 主席 习...,1
3,7104,习近平 出席 亚太经合组织 领导人 东道主 嘉宾 非正式 对话会 暨 工作 午宴 央视网 2...,1
4,2427,中国 国家 主席 习近平 11 14 日应 美国 总统 拜登 邀请 赴美 中 美 领导人 会...,1
...,...,...,...
3400,1044,从头到尾 来讲 一遍 惯例 先 谈谈 缅甸 北部 那场 风波 这场 风波 源头 藏匿在 缅甸...,23
3401,9043,缅北 全线 激战 敏昂 莱 誓言 反击 冲突 扩大化 中 缅 边境 乱 几天 缅甸 内战 爆...,23
3402,5570,近日 缅甸 内战 再次 卷土重来 国际 社会 关注 这次 冲突 异常 纷乱 消息 传来 人 ...,23
3403,5523,缅北 战火 越烧 越大 中方 做 两手 高层 去 缅甸 首都 清酒 半栩换种 角度 分析 故...,23


In [15]:
LOWERCASE = False
VOCAB_SIZE = 100_000

In [16]:
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

dataset = Dataset.from_pandas(test[['content']])


def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i: i + 1000]["content"]


raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(test['content'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['content'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

100%|██████████| 6803/6803 [00:06<00:00, 1013.00it/s]


In [17]:
def dummy(text):
    return text
vectorizer = TfidfVectorizer(lowercase=False, sublinear_tf=True, analyzer='word',
                             tokenizer=dummy,
                             preprocessor=dummy,
                             token_pattern=None, strip_accents='unicode'
                             )

vectorizer.fit(tokenized_texts_test)

vocab = vectorizer.vocabulary_

vectorizer = TfidfVectorizer(lowercase=False, sublinear_tf=True, vocabulary=vocab,
                             analyzer='word',
                             tokenizer=dummy,
                             preprocessor=dummy,
                             token_pattern=None, strip_accents='unicode'
                             )

X_train = vectorizer.fit_transform(tokenized_texts_train)
joblib.dump(vectorizer, 'vectorizer.pkl')
print(len(vocab))

89684


In [18]:
clf = MultinomialNB(alpha=0.02)
sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber")
p6 = {
    'n_iter': 1500, 
    'verbose': -1, 
    'objective': 'multiclass', 
    'metric': 'multi_logloss', 
    'learning_rate': 0.05073909898961407,
    'colsample_bytree': 0.726023996436955, 
    'colsample_bynode': 0.5803681307354022, 
    'lambda_l1': 8.562963348932286,
    'lambda_l2': 4.893256185259296, 
    'min_data_in_leaf': 115, 
    'max_depth': 23, 
    'max_bin': 898,
    'num_class': 24
}
lgb = LGBMClassifier(**p6)

# Creating the ensemble model
ensemble = VotingClassifier(estimators=[
    ('mnb', clf),
    ('sgd', sgd_model),
    ('lgb', lgb)],
    weights=[0.1, 0.45, 0.45],
    voting='soft',
    n_jobs=-1)

In [19]:
Y_train = train['topic'].values
ensemble.fit(X_train, Y_train)
joblib.dump(ensemble, 'ensemble.pkl')

['ensemble.pkl']

In [20]:
tf_test = vectorizer.transform(tokenized_texts_test)
pred = ensemble.predict(tf_test)
prob = ensemble.predict_proba(tf_test)
test["pred"] = pred
test["prob"] = prob.max(axis=1)

C:\Users\shens\AppData\Local\Temp\ipykernel_79648\608644723.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pred"] = pred


In [21]:
different = test[test['topic'] != test['pred']]
different

,id,content,topic,pred,prob
0,181,2023 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人...,1,22,0.695260
7,2861,APEC 峰会 前 公使 提出 四点 核心 利益 中 美 会晤 看 美 表现 一带 一路 国...,1,4,0.633975
40,4395,2023 亚太经合 会 APEC 领袖 峰会 今 15 日于 美国 旧金山 登场 泰国 总理...,1,0,0.471726
43,3236,焦点访谈 把舵 中美关系 领航 亚太 合作 央视网 2023 11 19 22 01 加载 ...,1,4,0.855680
60,3670,亚太 国家 地区 领导人 13 抵达 美国 亚太经合组织 APEC 第三十次 领导人 非正式...,1,4,0.791514
...,...,...,...,...,...
3096,7668,2022 11 28 上午 9 时 神舟 十五号 载人 飞行 新闻 发布会 酒泉卫星发射中心...,21,19,0.952397
3099,598,2023 12 31 日晚 国家 主席 习近平 发表 二 〇 二 四年 新年贺词 2013 ...,21,19,0.493249
3127,5424,神舟 十七号 发射成功 续写 飞天 梦想 建设 航天 强国 北京 时间 10 26 11 时...,21,20,0.958388
3132,6470,神舟 十六号 出舱 多个 首次 神 十七 10 发射 航天员 杨利伟 7 国外 航天 执行 ...,21,20,0.724031


In [22]:
accuracy = accuracy_score(test['topic'], test['pred'])
precision = precision_score(test['topic'], test['pred'], average='weighted')
recall = recall_score(test['topic'], test['pred'], average='weighted')
f1 = f1_score(test['topic'], test['pred'], average='weighted')
print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1: {f1}')

Accuracy: 0.9753303964757709
Precision: 0.9754709534500239
Recall: 0.9753303964757709
F1: 0.9753044435467418
